# Example for:
Transfer Learning Empirical Experiment from all 10 classes of MNIST into all 10 classes of FashionMNIST \
Note that here we are using the same architecture we used in FashionMNIST folder, with 5 CNN layers and 1 linear layer

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 5
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5.6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64,
      'kernel_size': 3,
      'activation_function': nn.ReLU,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001, 
      'num_train': 40,
      'early_stop_patience': 6,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': list(range(10)),
      'fine_tune_classes': list(range(10)),
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      # default value for pooling_every_n_layers is 1 (after each cnn layer)
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True         # CHANGE: reinit the conv lyers only after the cut
      }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.MNIST, datasets.FashionMNIST, root_dir, transform=transform)

In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

54000

## Pretraining

In [6]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=10, bias=True)
)

In [9]:
# Train and evaluate
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 47.74%
Validation Accuracy: 47.67%
Epoch: 1 	Training Accuracy: 75.33%
Validation Accuracy: 74.87%
Epoch: 2 	Training Accuracy: 82.54%
Validation Accuracy: 81.30%
Epoch: 3 	Training Accuracy: 87.35%
Validation Accuracy: 86.38%
Epoch: 4 	Training Accuracy: 89.96%
Validation Accuracy: 89.22%
Epoch: 5 	Training Accuracy: 92.27%
Validation Accuracy: 91.60%
Epoch: 6 	Training Accuracy: 93.92%
Validation Accuracy: 93.58%
Epoch: 7 	Training Accuracy: 95.48%
Validation Accuracy: 95.22%
Epoch: 8 	Training Accuracy: 95.81%
Validation Accuracy: 95.62%
Epoch: 9 	Training Accuracy: 94.51%
Validation Accuracy: 94.42%
Epoch: 10 	Training Accuracy: 96.38%
Validation Accuracy: 96.10%
Epoch: 11 	Training Accuracy: 97.17%
Validation Accuracy: 96.97%
Epoch: 12 	Training Accuracy: 97.34%
Validation Accuracy: 97.03%
Epoch: 13 	Training Accuracy: 97.67%
Validation Accuracy: 97.30%
Epoch: 14 	Training Accuracy: 97.27%
Validation Accuracy: 97.13%
Epoch: 15 	Training Accuracy: 97.97

In [10]:
#torch.save(pretrained_model.state_dict(), 'pretrained_models/pretrained_model_only_numbers_new.pth')

In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_model_only_numbers_new.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=10, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.test_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.0307, Accuracy: 59436.0/60000 (99%)

              precision    recall  f1-score   support

     Class 0       0.99      1.00      1.00      5923
     Class 1       0.99      1.00      0.99      6742
     Class 2       0.99      0.99      0.99      5958
     Class 3       0.99      0.99      0.99      6131
     Class 4       0.99      0.99      0.99      5842
     Class 5       0.99      0.99      0.99      5421
     Class 6       0.99      0.99      0.99      5918
     Class 7       0.99      1.00      0.99      6265
     Class 8       0.99      0.98      0.99      5851
     Class 9       0.98      0.99      0.98      5949

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000



0.9906

In [12]:
# save model for later use
foldername = "pretrained_models/pretrained_mnist_to_fashion_all_classes"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp.get("activation_function", nn.ReLU))
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

In [ ]:
# load results: to continue from a checkpoint (actually don't run)
with open('results.json', 'r') as f:
    results = json.load(f)

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [9]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0


In [ ]:
print(baselines_results)

In [20]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_baselines_mnisttofashion', 'w') as f:
    json.dump(baselines_results, f)

### Fine-tuning

In [9]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [1.0]

In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.8849, Test Accuracy: 0.8841

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Training Accuracy: 0.8917, Test Accuracy: 0.8905

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Training Accuracy: 0.8956, Test Accuracy: 0.8944

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Training Accuracy: 0.8917, Test Accuracy: 0.8903

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Training Accuracy: 0.8959, Test Accuracy: 0.8952

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Training Accuracy: 0.8989, Test Accuracy: 0.8972

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9019, Test Accuracy: 0.9008

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 2
Training Accuracy: 0.8954, Test Accuracy: 0.8946

Sampled Percentage: 1.0, Sample

TypeError: slice indices must be integers or None or have an __index__ method

In [11]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8848915524055142, 'test_acc': 0.88405}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8917379445980296, 'test_acc': 0.89055}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8955786036328552, 'test_acc': 0.8944333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8916822828728872, 'test_acc': 0.8902666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8959125739837097, 'test_acc': 0.8952166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8988811993246377, 'test_acc': 0.89725}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9019054863907082, 'test_acc': 0.9007666666666667}, {'lr': 0.001, 'sampled_p

In [12]:
cuts = [5,6]

In [13]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1.0, Sampled Cut Point: 5, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.8644, Test Accuracy: 0.8638

Sampled Percentage: 1.0, Sampled Cut Point: 5, Lr: 0.001, Repeat: 1
Training Accuracy: 0.8758, Test Accuracy: 0.8749

Sampled Percentage: 1.0, Sampled Cut Point: 5, Lr: 0.001, Repeat: 2
Training Accuracy: 0.8748, Test Accuracy: 0.8736

Sampled Percentage: 1.0, Sampled Cut Point: 5, Lr: 0.001, Repeat: 3
Training Accuracy: 0.8611, Test Accuracy: 0.8598

Sampled Percentage: 1.0, Sampled Cut Point: 5, Lr: 0.001, Repeat: 4
Training Accuracy: 0.8749, Test Accuracy: 0.8735

Sampled Percentage: 1.0, Sampled Cut Point: 6, Lr: 0.001, Repeat: 0
Training Accuracy: 0.8505, Test Accuracy: 0.8495

Sampled Percentage: 1.0, Sampled Cut Point: 6, Lr: 0.001, Repeat: 1
Training Accuracy: 0.8520, Test Accuracy: 0.8513

Sampled Percentage: 1.0, Sampled Cut Point: 6, Lr: 0.001, Repeat: 2
Training Accuracy: 0.8538, Test Accuracy: 0.8528

Sampled Percentage: 1.0, Sample

In [14]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8848915524055142, 'test_acc': 0.88405}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8917379445980296, 'test_acc': 0.89055}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8955786036328552, 'test_acc': 0.8944333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8916822828728872, 'test_acc': 0.8902666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8959125739837097, 'test_acc': 0.8952166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8988811993246377, 'test_acc': 0.89725}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9019054863907082, 'test_acc': 0.9007666666666667}, {'lr': 0.001, 'sampled_p

In [15]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8848915524055142, 'test_acc': 0.88405}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8917379445980296, 'test_acc': 0.89055}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8955786036328552, 'test_acc': 0.8944333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8916822828728872, 'test_acc': 0.8902666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8959125739837097, 'test_acc': 0.8952166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8988811993246377, 'test_acc': 0.89725}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9019054863907082, 'test_acc': 0.9007666666666667}, {'lr': 0.001, 'sampled_p

In [16]:
baselines_results = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.56, 'test_acc': 0.49546666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9, 'test_acc': 0.5880333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.76, 'test_acc': 0.5669}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.66, 'test_acc': 0.5566333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.64, 'test_acc': 0.50875}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.58, 'test_acc': 0.5023166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.84, 'test_acc': 0.6063833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.76, 'test_acc': 0.5173666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.82, 'test_acc': 0.5828833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.8, 'test_acc': 0.6151166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.92, 'test_acc': 0.6114333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.74, 'test_acc': 0.6201}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.68, 'test_acc': 0.5493333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.66, 'test_acc': 0.5475666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.9, 'test_acc': 0.5934}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.72, 'test_acc': 0.5225666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.9, 'test_acc': 0.5742833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.86, 'test_acc': 0.5807333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.86, 'test_acc': 0.5818}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.72, 'test_acc': 0.5782833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.92, 'test_acc': 0.6376}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.94, 'test_acc': 0.5610833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.4, 'test_acc': 0.36678333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.9, 'test_acc': 0.6216333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.88, 'test_acc': 0.601}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.762962962962963, 'test_acc': 0.73785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.5703703703703704, 'test_acc': 0.5428666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.687037037037037, 'test_acc': 0.6838333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.7129629629629629, 'test_acc': 0.6703}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.7166666666666667, 'test_acc': 0.6885666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.7222222222222222, 'test_acc': 0.7058}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.7666666666666667, 'test_acc': 0.7047}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.737037037037037, 'test_acc': 0.6762666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.7351851851851852, 'test_acc': 0.68785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.5518518518518518, 'test_acc': 0.5358166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.7666666666666667, 'test_acc': 0.7022166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.5296296296296297, 'test_acc': 0.5434166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.7148148148148148, 'test_acc': 0.6945}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.5314814814814814, 'test_acc': 0.5411666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.6462962962962963, 'test_acc': 0.6158333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.7685185185185185, 'test_acc': 0.7105}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.7148148148148148, 'test_acc': 0.6789666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.7944444444444444, 'test_acc': 0.7026833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.7611111111111111, 'test_acc': 0.7129333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.6944444444444444, 'test_acc': 0.6863666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.737037037037037, 'test_acc': 0.6806666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.7574074074074074, 'test_acc': 0.7051666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.5722222222222222, 'test_acc': 0.5656166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.5777777777777777, 'test_acc': 0.5531666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.737037037037037, 'test_acc': 0.6950333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8092592592592592, 'test_acc': 0.79295}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.7905555555555556, 'test_acc': 0.7855}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.7477777777777778, 'test_acc': 0.7487333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.7701851851851852, 'test_acc': 0.76665}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.7753703703703704, 'test_acc': 0.7688833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.7855555555555556, 'test_acc': 0.7838333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.7787037037037037, 'test_acc': 0.7705}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.7735185185185185, 'test_acc': 0.7637833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.7635185185185185, 'test_acc': 0.7632333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.7670370370370371, 'test_acc': 0.7599}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.7842592592592592, 'test_acc': 0.7750333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.7737037037037037, 'test_acc': 0.7611833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.7648148148148148, 'test_acc': 0.7529666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.7798148148148148, 'test_acc': 0.77445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.77, 'test_acc': 0.7596}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.7796296296296297, 'test_acc': 0.7742}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.7848148148148149, 'test_acc': 0.7725833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.7868518518518518, 'test_acc': 0.7808833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.7888888888888889, 'test_acc': 0.7816666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.7587037037037037, 'test_acc': 0.7624166666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8722962962962963, 'test_acc': 0.8664166666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.8705555555555555, 'test_acc': 0.8673833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.8762962962962964, 'test_acc': 0.8701}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.8655925925925926, 'test_acc': 0.8620666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8631481481481481, 'test_acc': 0.8585166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8848915524055142, 'test_acc': 0.88405}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.8917379445980296, 'test_acc': 0.89055}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.8955786036328552, 'test_acc': 0.8944333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.8916822828728872, 'test_acc': 0.8902666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8959125739837097, 'test_acc': 0.8952166666666667}]
finetune1 = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.56, 'test_acc': 0.49546666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9, 'test_acc': 0.5880333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.76, 'test_acc': 0.5669}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.66, 'test_acc': 0.5566333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.64, 'test_acc': 0.50875}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.58, 'test_acc': 0.5023166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.84, 'test_acc': 0.6063833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.76, 'test_acc': 0.5173666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.82, 'test_acc': 0.5828833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.8, 'test_acc': 0.6151166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.92, 'test_acc': 0.6114333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.74, 'test_acc': 0.6201}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.68, 'test_acc': 0.5493333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.66, 'test_acc': 0.5475666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.9, 'test_acc': 0.5934}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.72, 'test_acc': 0.5225666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.9, 'test_acc': 0.5742833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.86, 'test_acc': 0.5807333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.86, 'test_acc': 0.5818}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.72, 'test_acc': 0.5782833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.92, 'test_acc': 0.6376}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.94, 'test_acc': 0.5610833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.4, 'test_acc': 0.36678333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.9, 'test_acc': 0.6216333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.88, 'test_acc': 0.601}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.98, 'test_acc': 0.5824333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.92, 'test_acc': 0.6029166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.78, 'test_acc': 0.5413}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.88, 'test_acc': 0.5822166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.8, 'test_acc': 0.5307333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.82, 'test_acc': 0.5573166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.96, 'test_acc': 0.6027166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.86, 'test_acc': 0.55455}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.88, 'test_acc': 0.6101666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.72, 'test_acc': 0.5593333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.96, 'test_acc': 0.6324666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.94, 'test_acc': 0.6398}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.76, 'test_acc': 0.51825}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.72, 'test_acc': 0.5638166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.92, 'test_acc': 0.6147}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.76, 'test_acc': 0.5700666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.82, 'test_acc': 0.5665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.5743333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.98, 'test_acc': 0.5502666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.76, 'test_acc': 0.5632333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.96, 'test_acc': 0.6617166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.74, 'test_acc': 0.5212333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.84, 'test_acc': 0.58445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.92, 'test_acc': 0.59035}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.9, 'test_acc': 0.592}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.78, 'test_acc': 0.5781166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.6}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.74, 'test_acc': 0.5148833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.86, 'test_acc': 0.558}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.54, 'test_acc': 0.48418333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.5631333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.92, 'test_acc': 0.607}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.84, 'test_acc': 0.5206666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.84, 'test_acc': 0.5857333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.78, 'test_acc': 0.6112333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.94, 'test_acc': 0.6016666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.92, 'test_acc': 0.6001833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.8, 'test_acc': 0.5345166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.66, 'test_acc': 0.5326666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.5626833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.5707}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.78, 'test_acc': 0.5406166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.58, 'test_acc': 0.50335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.96, 'test_acc': 0.53855}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.82, 'test_acc': 0.5561666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.92, 'test_acc': 0.6286666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.9, 'test_acc': 0.55445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.84, 'test_acc': 0.5846166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.98, 'test_acc': 0.6007166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.82, 'test_acc': 0.5896666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.78, 'test_acc': 0.58835}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.98, 'test_acc': 0.60575}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.8, 'test_acc': 0.5703333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.8, 'test_acc': 0.5840166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.98, 'test_acc': 0.5110666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.96, 'test_acc': 0.6023333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9, 'test_acc': 0.6172666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.82, 'test_acc': 0.5309666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.98, 'test_acc': 0.5793833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.8, 'test_acc': 0.63255}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.64, 'test_acc': 0.54635}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.9, 'test_acc': 0.6294833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.7, 'test_acc': 0.5315333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.78, 'test_acc': 0.52305}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.88, 'test_acc': 0.5502166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.94, 'test_acc': 0.5872166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.84, 'test_acc': 0.5684666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.96, 'test_acc': 0.5704666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.8, 'test_acc': 0.5698666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.64, 'test_acc': 0.55895}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6306166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.74, 'test_acc': 0.549}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.88, 'test_acc': 0.58815}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.88, 'test_acc': 0.61305}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.9, 'test_acc': 0.5930833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.88, 'test_acc': 0.5812833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.96, 'test_acc': 0.6032666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.86, 'test_acc': 0.58605}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.98, 'test_acc': 0.5996}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.84, 'test_acc': 0.5441166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.9, 'test_acc': 0.6063833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.9, 'test_acc': 0.6088666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.94, 'test_acc': 0.5739833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.6278}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.92, 'test_acc': 0.6359666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.98, 'test_acc': 0.6051166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.86, 'test_acc': 0.611}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.94, 'test_acc': 0.5748833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.68, 'test_acc': 0.5347}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.5694333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.84, 'test_acc': 0.5908}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.86, 'test_acc': 0.5998166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.9, 'test_acc': 0.5614166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.82, 'test_acc': 0.5656166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.88, 'test_acc': 0.5744333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6615666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.9, 'test_acc': 0.5737333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.74, 'test_acc': 0.5704}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.8, 'test_acc': 0.6053666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.9, 'test_acc': 0.6030166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.92, 'test_acc': 0.6207666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.98, 'test_acc': 0.5960666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.74, 'test_acc': 0.5606333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.6250166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.88, 'test_acc': 0.5427333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.92, 'test_acc': 0.6087333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.86, 'test_acc': 0.6153833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.98, 'test_acc': 0.5967}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.6108833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.98, 'test_acc': 0.6216833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.5961666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9, 'test_acc': 0.6153}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.6027166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.5725666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.5743666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.98, 'test_acc': 0.6065166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.96, 'test_acc': 0.5933833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.88, 'test_acc': 0.5620333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.92, 'test_acc': 0.5717}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.94, 'test_acc': 0.5657166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6506}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.94, 'test_acc': 0.5826166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.94, 'test_acc': 0.5932}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.98, 'test_acc': 0.6437833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.82, 'test_acc': 0.60835}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.86, 'test_acc': 0.5628}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.94, 'test_acc': 0.5951333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.96, 'test_acc': 0.5536}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.57145}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.98, 'test_acc': 0.5107833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.98, 'test_acc': 0.5911666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.5776833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.92, 'test_acc': 0.5956}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.5744}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.98, 'test_acc': 0.6112666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.5928333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.82, 'test_acc': 0.5652333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.98, 'test_acc': 0.5938166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.86, 'test_acc': 0.5493333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.5804333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.5755}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.96, 'test_acc': 0.5738333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.94, 'test_acc': 0.5654333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.96, 'test_acc': 0.5512}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.9, 'test_acc': 0.5204333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6191666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.5858333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.86, 'test_acc': 0.5636833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.9, 'test_acc': 0.61835}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.92, 'test_acc': 0.5408833333333334}]
finetune2 = [{'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.762962962962963, 'test_acc': 0.73785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.5703703703703704, 'test_acc': 0.5428666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.687037037037037, 'test_acc': 0.6838333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.7129629629629629, 'test_acc': 0.6703}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.7166666666666667, 'test_acc': 0.6885666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.7222222222222222, 'test_acc': 0.7058}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.7666666666666667, 'test_acc': 0.7047}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.737037037037037, 'test_acc': 0.6762666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.7351851851851852, 'test_acc': 0.68785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.5518518518518518, 'test_acc': 0.5358166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.7666666666666667, 'test_acc': 0.7022166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.5296296296296297, 'test_acc': 0.5434166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.7148148148148148, 'test_acc': 0.6945}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.5314814814814814, 'test_acc': 0.5411666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.6462962962962963, 'test_acc': 0.6158333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.7685185185185185, 'test_acc': 0.7105}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.7148148148148148, 'test_acc': 0.6789666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.7944444444444444, 'test_acc': 0.7026833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.7611111111111111, 'test_acc': 0.7129333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.6944444444444444, 'test_acc': 0.6863666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.737037037037037, 'test_acc': 0.6806666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.7574074074074074, 'test_acc': 0.7051666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.5722222222222222, 'test_acc': 0.5656166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.5777777777777777, 'test_acc': 0.5531666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.737037037037037, 'test_acc': 0.6950333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.7074074074074074, 'test_acc': 0.7167833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.7703703703703704, 'test_acc': 0.7097666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.7425925925925926, 'test_acc': 0.7175333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.562962962962963, 'test_acc': 0.57505}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.7111111111111111, 'test_acc': 0.6827333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.7240740740740741, 'test_acc': 0.6879333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.6111111111111112, 'test_acc': 0.6047833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.7777777777777778, 'test_acc': 0.7205666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.7537037037037037, 'test_acc': 0.6994333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.7148148148148148, 'test_acc': 0.6797166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.7555555555555555, 'test_acc': 0.7230166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.5685185185185185, 'test_acc': 0.5829}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.7018518518518518, 'test_acc': 0.6828666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.7481481481481481, 'test_acc': 0.7001666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.7407407407407407, 'test_acc': 0.70065}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.7814814814814814, 'test_acc': 0.7121166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.7777777777777778, 'test_acc': 0.7264333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.7925925925925926, 'test_acc': 0.702}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.7555555555555555, 'test_acc': 0.70035}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.6796296296296296, 'test_acc': 0.6698166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.7777777777777778, 'test_acc': 0.7288}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.7333333333333333, 'test_acc': 0.6923}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.7555555555555555, 'test_acc': 0.7165166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.562962962962963, 'test_acc': 0.5236}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.7203703703703703, 'test_acc': 0.6865}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.7351851851851852, 'test_acc': 0.6885}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.7518518518518519, 'test_acc': 0.6869666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.725925925925926, 'test_acc': 0.6887166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.7518518518518519, 'test_acc': 0.68255}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.7185185185185186, 'test_acc': 0.6811166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.7148148148148148, 'test_acc': 0.6885}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.7333333333333333, 'test_acc': 0.6851166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.7481481481481481, 'test_acc': 0.6796166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.7537037037037037, 'test_acc': 0.6766833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.7444444444444445, 'test_acc': 0.6801666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.7592592592592593, 'test_acc': 0.70005}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.7055555555555556, 'test_acc': 0.6663166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.762962962962963, 'test_acc': 0.7051333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.7314814814814815, 'test_acc': 0.66865}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.7518518518518519, 'test_acc': 0.6795833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.7407407407407407, 'test_acc': 0.6694666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.7148148148148148, 'test_acc': 0.6652666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.7685185185185185, 'test_acc': 0.6835}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.774074074074074, 'test_acc': 0.69955}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.6796296296296296, 'test_acc': 0.6692166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.7666666666666667, 'test_acc': 0.7075833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.7222222222222222, 'test_acc': 0.6624333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.7685185185185185, 'test_acc': 0.6970166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.7703703703703704, 'test_acc': 0.7189666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.75, 'test_acc': 0.68125}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.7759259259259259, 'test_acc': 0.7220666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.7888888888888889, 'test_acc': 0.7062833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.7518518518518519, 'test_acc': 0.7031}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.7685185185185185, 'test_acc': 0.6924166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.7444444444444445, 'test_acc': 0.7162}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.7537037037037037, 'test_acc': 0.7119333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.774074074074074, 'test_acc': 0.7174666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.8092592592592592, 'test_acc': 0.7176333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.7518518518518519, 'test_acc': 0.6892166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.5444444444444444, 'test_acc': 0.55885}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.774074074074074, 'test_acc': 0.7069}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.7925925925925926, 'test_acc': 0.7151666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.55, 'test_acc': 0.5568666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.774074074074074, 'test_acc': 0.71095}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.7703703703703704, 'test_acc': 0.7082833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.7944444444444444, 'test_acc': 0.7035833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.5888888888888889, 'test_acc': 0.55685}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.5888888888888889, 'test_acc': 0.5754166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.575925925925926, 'test_acc': 0.5587}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.7722222222222223, 'test_acc': 0.7123}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.7851851851851852, 'test_acc': 0.7148166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.774074074074074, 'test_acc': 0.69335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.7777777777777778, 'test_acc': 0.7079}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.5722222222222222, 'test_acc': 0.54145}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.8, 'test_acc': 0.7306}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8351851851851851, 'test_acc': 0.7313333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8351851851851851, 'test_acc': 0.7377166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8240740740740741, 'test_acc': 0.7351833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.8296296296296296, 'test_acc': 0.73235}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8222222222222222, 'test_acc': 0.7381166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.8407407407407408, 'test_acc': 0.7432333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.8111111111111111, 'test_acc': 0.7440666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8425925925925926, 'test_acc': 0.7402333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8240740740740741, 'test_acc': 0.7387}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.7925925925925926, 'test_acc': 0.7177666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.825925925925926, 'test_acc': 0.7309333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.7481481481481481, 'test_acc': 0.6943833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8148148148148148, 'test_acc': 0.7449333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.8092592592592592, 'test_acc': 0.7265333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8574074074074074, 'test_acc': 0.7339833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.8333333333333334, 'test_acc': 0.7251833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.8148148148148148, 'test_acc': 0.7306333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.8462962962962963, 'test_acc': 0.7351833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.837037037037037, 'test_acc': 0.73475}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8351851851851851, 'test_acc': 0.72925}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.8222222222222222, 'test_acc': 0.7313166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.8407407407407408, 'test_acc': 0.73085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.8296296296296296, 'test_acc': 0.73685}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.8148148148148148, 'test_acc': 0.7450833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.837037037037037, 'test_acc': 0.7482333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8518518518518519, 'test_acc': 0.7451166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.8462962962962963, 'test_acc': 0.7377166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.7981481481481482, 'test_acc': 0.7269666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.7981481481481482, 'test_acc': 0.7213666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8277777777777777, 'test_acc': 0.7416833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.8518518518518519, 'test_acc': 0.7493333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.8092592592592592, 'test_acc': 0.72535}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.8648148148148148, 'test_acc': 0.7503}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.8462962962962963, 'test_acc': 0.7331833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.8407407407407408, 'test_acc': 0.7323}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.8870370370370371, 'test_acc': 0.7470833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.8555555555555555, 'test_acc': 0.7354833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8166666666666667, 'test_acc': 0.74085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.8203703703703704, 'test_acc': 0.7189}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.8444444444444444, 'test_acc': 0.7306}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8351851851851851, 'test_acc': 0.7211666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.8314814814814815, 'test_acc': 0.73615}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.8648148148148148, 'test_acc': 0.7376}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.8148148148148148, 'test_acc': 0.7287}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.8703703703703703, 'test_acc': 0.74195}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.8481481481481481, 'test_acc': 0.7467833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.8018518518518518, 'test_acc': 0.73105}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.8537037037037037, 'test_acc': 0.7431833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.8314814814814815, 'test_acc': 0.7484833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.8333333333333334, 'test_acc': 0.7449833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.7907407407407407, 'test_acc': 0.7161166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.7685185185185185, 'test_acc': 0.6822666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.7907407407407407, 'test_acc': 0.7224833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.7388888888888889, 'test_acc': 0.6737333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.7777777777777778, 'test_acc': 0.7055}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.7574074074074074, 'test_acc': 0.7082333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.7351851851851852, 'test_acc': 0.6717333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.7777777777777778, 'test_acc': 0.7022166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.7722222222222223, 'test_acc': 0.69005}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.7925925925925926, 'test_acc': 0.6948}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.7888888888888889, 'test_acc': 0.7028833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.7907407407407407, 'test_acc': 0.6928}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.7722222222222223, 'test_acc': 0.7018833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.75, 'test_acc': 0.6873}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.7685185185185185, 'test_acc': 0.7041833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.7944444444444444, 'test_acc': 0.6988333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.7944444444444444, 'test_acc': 0.6874}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.8, 'test_acc': 0.6980166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.7777777777777778, 'test_acc': 0.6955}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.7944444444444444, 'test_acc': 0.69185}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.7666666666666667, 'test_acc': 0.69925}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.7703703703703704, 'test_acc': 0.701}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.774074074074074, 'test_acc': 0.6941833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.7574074074074074, 'test_acc': 0.70195}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.7222222222222222, 'test_acc': 0.68955}]
finetune3 = [{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8092592592592592, 'test_acc': 0.79295}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.7905555555555556, 'test_acc': 0.7855}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.7477777777777778, 'test_acc': 0.7487333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.7701851851851852, 'test_acc': 0.76665}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.7753703703703704, 'test_acc': 0.7688833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.7855555555555556, 'test_acc': 0.7838333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.7787037037037037, 'test_acc': 0.7705}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.7735185185185185, 'test_acc': 0.7637833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.7635185185185185, 'test_acc': 0.7632333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.7670370370370371, 'test_acc': 0.7599}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.7842592592592592, 'test_acc': 0.7750333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.7737037037037037, 'test_acc': 0.7611833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.7648148148148148, 'test_acc': 0.7529666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.7798148148148148, 'test_acc': 0.77445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.77, 'test_acc': 0.7596}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.7796296296296297, 'test_acc': 0.7742}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.7848148148148149, 'test_acc': 0.7725833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.7868518518518518, 'test_acc': 0.7808833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.7888888888888889, 'test_acc': 0.7816666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.7587037037037037, 'test_acc': 0.7624166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.7855555555555556, 'test_acc': 0.7735166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.785, 'test_acc': 0.7805}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.7735185185185185, 'test_acc': 0.7669166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.7888888888888889, 'test_acc': 0.78315}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.78, 'test_acc': 0.7725833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.7818518518518518, 'test_acc': 0.7773833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.7588888888888888, 'test_acc': 0.7510833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.7874074074074074, 'test_acc': 0.7772166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.7625925925925926, 'test_acc': 0.7586}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.7762962962962963, 'test_acc': 0.7738833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.7888888888888889, 'test_acc': 0.7778833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.7892592592592592, 'test_acc': 0.7777666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.78, 'test_acc': 0.7691833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.7687037037037037, 'test_acc': 0.7655333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.7592592592592593, 'test_acc': 0.7495833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.7768518518518519, 'test_acc': 0.7670333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.7931481481481482, 'test_acc': 0.7788333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.7824074074074074, 'test_acc': 0.7692166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.79, 'test_acc': 0.7816833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.7770370370370371, 'test_acc': 0.7815}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.787962962962963, 'test_acc': 0.7811}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.792962962962963, 'test_acc': 0.7769333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.7798148148148148, 'test_acc': 0.7724}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.7781481481481481, 'test_acc': 0.7740833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8012962962962963, 'test_acc': 0.7908666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.7592592592592593, 'test_acc': 0.7478333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.7577777777777778, 'test_acc': 0.74965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.7927777777777778, 'test_acc': 0.77605}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.7831481481481481, 'test_acc': 0.7760666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.7853703703703704, 'test_acc': 0.7732}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.765, 'test_acc': 0.7576333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.7659259259259259, 'test_acc': 0.7565666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.797962962962963, 'test_acc': 0.7838666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.7570370370370371, 'test_acc': 0.7488833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.7648148148148148, 'test_acc': 0.75555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.7607407407407407, 'test_acc': 0.7515666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.7888888888888889, 'test_acc': 0.7753833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.7481481481481481, 'test_acc': 0.7453333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.8, 'test_acc': 0.7880166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.7696296296296297, 'test_acc': 0.7680666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8151851851851852, 'test_acc': 0.7965666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.7992592592592592, 'test_acc': 0.7879333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.802962962962963, 'test_acc': 0.78865}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.7788888888888889, 'test_acc': 0.7726166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.7805555555555556, 'test_acc': 0.76725}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.7996296296296296, 'test_acc': 0.7911833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.8016666666666666, 'test_acc': 0.7873}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.7894444444444444, 'test_acc': 0.7742}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.797962962962963, 'test_acc': 0.7861166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.8001851851851852, 'test_acc': 0.7858833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.8111111111111111, 'test_acc': 0.7940833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.8107407407407408, 'test_acc': 0.78975}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.8007407407407408, 'test_acc': 0.7809166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.8012962962962963, 'test_acc': 0.7891666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.8061111111111111, 'test_acc': 0.7917333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.7959259259259259, 'test_acc': 0.7869666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.8009259259259259, 'test_acc': 0.7848333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.7918518518518518, 'test_acc': 0.7841666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.799074074074074, 'test_acc': 0.7876666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.7942592592592592, 'test_acc': 0.7869}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8044444444444444, 'test_acc': 0.7870666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8144444444444444, 'test_acc': 0.7999666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8190740740740741, 'test_acc': 0.8012}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.815, 'test_acc': 0.8008833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8235185185185185, 'test_acc': 0.80865}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.7868518518518518, 'test_acc': 0.7818666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.804074074074074, 'test_acc': 0.795}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8262962962962963, 'test_acc': 0.8084166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8268518518518518, 'test_acc': 0.8072666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.8062962962962963, 'test_acc': 0.7897833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.8253703703703704, 'test_acc': 0.8043333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8138888888888889, 'test_acc': 0.7977833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8166666666666667, 'test_acc': 0.7965666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.8107407407407408, 'test_acc': 0.7998166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8272222222222222, 'test_acc': 0.8106166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.8151851851851852, 'test_acc': 0.8035166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.8174074074074074, 'test_acc': 0.8057666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.8122222222222222, 'test_acc': 0.79945}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.81, 'test_acc': 0.7949}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.812962962962963, 'test_acc': 0.7975}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8322222222222222, 'test_acc': 0.8104333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.8311111111111111, 'test_acc': 0.8091}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.8144444444444444, 'test_acc': 0.7972333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.8005555555555556, 'test_acc': 0.7908166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8294444444444444, 'test_acc': 0.8122166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.8290740740740741, 'test_acc': 0.8110166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.8037037037037037, 'test_acc': 0.7952166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.8294444444444444, 'test_acc': 0.8098166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.8290740740740741, 'test_acc': 0.8104666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.8314814814814815, 'test_acc': 0.8116}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.8362962962962963, 'test_acc': 0.8170166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.8314814814814815, 'test_acc': 0.8082666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8233333333333334, 'test_acc': 0.8028166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.8187037037037037, 'test_acc': 0.79965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.8262962962962963, 'test_acc': 0.80615}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8172222222222222, 'test_acc': 0.7985166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.8175925925925925, 'test_acc': 0.7986666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.8205555555555556, 'test_acc': 0.8095166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.8122222222222222, 'test_acc': 0.7959166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.8268518518518518, 'test_acc': 0.8125}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.7955555555555556, 'test_acc': 0.7807}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.7846296296296297, 'test_acc': 0.7671333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8022222222222222, 'test_acc': 0.7834166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.7712962962962963, 'test_acc': 0.7651166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.7933333333333333, 'test_acc': 0.7804833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.7812962962962963, 'test_acc': 0.7747666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.7561111111111111, 'test_acc': 0.7536166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.7859259259259259, 'test_acc': 0.7710166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.7796296296296297, 'test_acc': 0.7735333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.792037037037037, 'test_acc': 0.7782166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.7820370370370371, 'test_acc': 0.7750833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.777962962962963, 'test_acc': 0.7667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.7842592592592592, 'test_acc': 0.7726}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.7848148148148149, 'test_acc': 0.7716}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.7964814814814815, 'test_acc': 0.7787333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.7955555555555556, 'test_acc': 0.781}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.7911111111111111, 'test_acc': 0.7763333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.79, 'test_acc': 0.7791166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.7822222222222223, 'test_acc': 0.772}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.7748148148148148, 'test_acc': 0.7680166666666667}]
finetune4 = [{'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8722962962962963, 'test_acc': 0.8664166666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8705555555555555, 'test_acc': 0.8673833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8762962962962964, 'test_acc': 0.8701}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8655925925925926, 'test_acc': 0.8620666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8631481481481481, 'test_acc': 0.8585166666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8605185185185186, 'test_acc': 0.8541333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.8693703703703703, 'test_acc': 0.8658666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.8730740740740741, 'test_acc': 0.86815}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.867, 'test_acc': 0.86105}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.8681111111111111, 'test_acc': 0.8634666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8674814814814815, 'test_acc': 0.86045}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8686666666666667, 'test_acc': 0.8646833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.8706296296296296, 'test_acc': 0.8638833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.8694074074074074, 'test_acc': 0.8653}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8728148148148148, 'test_acc': 0.8671333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8655555555555555, 'test_acc': 0.8563333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8751851851851852, 'test_acc': 0.8686833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.8768888888888889, 'test_acc': 0.8675833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.8717407407407407, 'test_acc': 0.8661333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.8736296296296296, 'test_acc': 0.8674333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8712962962962963, 'test_acc': 0.86285}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8662962962962963, 'test_acc': 0.85965}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8781481481481481, 'test_acc': 0.8700666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.8696666666666667, 'test_acc': 0.8644}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8735925925925926, 'test_acc': 0.8658166666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8649259259259259, 'test_acc': 0.8557666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.8646666666666667, 'test_acc': 0.8582666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.8647407407407407, 'test_acc': 0.85635}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.8542592592592593, 'test_acc': 0.8480166666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8559629629629629, 'test_acc': 0.8501666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8408148148148148, 'test_acc': 0.8328833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8376296296296296, 'test_acc': 0.83375}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8428518518518519, 'test_acc': 0.8349166666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8344444444444444, 'test_acc': 0.8301833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8353333333333334, 'test_acc': 0.8308333333333333}]
finetune5 = [{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8848915524055142, 'test_acc': 0.88405}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8917379445980296, 'test_acc': 0.89055}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8955786036328552, 'test_acc': 0.8944333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8916822828728872, 'test_acc': 0.8902666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8959125739837097, 'test_acc': 0.8952166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8988811993246377, 'test_acc': 0.89725}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9019054863907082, 'test_acc': 0.9007666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.8953745106406664, 'test_acc': 0.8946}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.8954672801825705, 'test_acc': 0.8949166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.8918307141399335, 'test_acc': 0.8912333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.887322114403399, 'test_acc': 0.8865}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8884539028146279, 'test_acc': 0.88755}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.8913112047052711, 'test_acc': 0.8905333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.8940015214204873, 'test_acc': 0.8925333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8881199324637735, 'test_acc': 0.8876333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8916266211477448, 'test_acc': 0.8905333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8846503515965638, 'test_acc': 0.8840166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.8804571683025029, 'test_acc': 0.8789666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.8840009648032358, 'test_acc': 0.8831}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.8692691615488802, 'test_acc': 0.8686833333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8789357478152773, 'test_acc': 0.8779333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8875818691207302, 'test_acc': 0.8864833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8882127020056775, 'test_acc': 0.8870833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.8791398408074661, 'test_acc': 0.87805}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8803458448522181, 'test_acc': 0.8794333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8644451453698722, 'test_acc': 0.8637833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.8758372451156836, 'test_acc': 0.87485}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.8748167801547396, 'test_acc': 0.8736333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.8610868879529473, 'test_acc': 0.8598333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8749281036050244, 'test_acc': 0.8735166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8504926062675102, 'test_acc': 0.8494833333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8520325806631167, 'test_acc': 0.8513166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8538323097760543, 'test_acc': 0.8528}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8500658663747518, 'test_acc': 0.8491}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8522181197469247, 'test_acc': 0.8513333333333334}]

In [19]:
results = finetune1 + finetune2 + finetune3 + finetune4 + finetune5

In [21]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_results_mnisttofashion.json', 'w') as f:
    json.dump(results, f)
results = results[1:]